
Импорт пакетов



In [42]:
import numpy as np
import math
from functools import reduce
from copy import deepcopy
import matplotlib.pyplot as plt
import numba as nb

Вентили 

In [43]:
H = np.array([[1/math.sqrt(2),1/math.sqrt(2)],[1/math.sqrt(2),-1/math.sqrt(2)]])
X = np.array([[0,1],[1,0]])
I = np.array([[1,0],[0,1]])

In [44]:
CNOT = np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])

Функция унитарного оператора действующего на 1 кубит

In [344]:
def UnitGate(q, matrix, q0):
  qubit = deepcopy(q)
  for i in range(len(qubit)):
    if (i % (1<<(q0+1))) < (1<<(q0)):
      r = 0
    else:
      r = 1
    qubit[i] = q[i|(1<<q0)] * matrix[r][1] + q[(i|(1<<q0))^(1<<q0)] * matrix[r][0]
  return qubit

Функция унитарного оператора действующего на 2 кубита

In [345]:
def UnitGatetwo(q,matrix,q1,q0):
  qubit = deepcopy(q)
  for i in range(len(q)):
    if (i % (1<<(q0+1))) < (1<<(q0)):
      r1 = 0
    else:
      r1 = 1
    if (i %(1<<(q1+1))) < (1<<(q1)):
      r2 = 0
    else:
      r2 = 1
    qubit[i] = q[(i|(1<<q0))|(1<<q1)] * matrix[2*r2 +r1][3]  \
    + q[((i|(1<<q0))|(1<<q1))^(1<<q0)] * matrix[2*r2 +r1][2] \
    + q[((i|(1<<q0))|(1<<q1))^(1<<q1)] * matrix[2*r2 +r1][1] \
    + q[(((i|(1<<q0)|(1<<q1))^(1<<q1))^(1<<q0))] * matrix[2*r2 +r1][0]
  return qubit

Вывод битовой строчки по числу

In [346]:
# Перевод i кубита по счету в битовую запись размера n
# Ввод (n) - количество кубит, (i) - номер кубита 
# вывод int запись бита
def inttobit(n, i):
    return list(map(int, list(format(i, "0{0}b".format(n)))))

Оракул константной функции

In [316]:
def constOracul(q,n):
  randomconst = np.random.randint(2)
  if randomconst == 1:
    return UnitGate(q, X, n)
  else:
    return q

Оракул сбалансированной функции

In [307]:
def balanceOracul(q,n):
  randombalance = np.random.randint(1,2**n)
  randombalancebit = inttobit(n, randombalance)
  q2 = deepcopy(q)
  for i in range(n):
    if randombalancebit[i] == 1:
      q2 = UnitGate(q2, X, i)
  for j in range(n):
    q2 = UnitGatetwo(q2,CNOT,j, n)
  for i in range(n):
    if randombalancebit[i] == 1:
      q2 = UnitGate(q2, X, i)
  return q2


Ввод количества входных кубит

In [347]:
n = int(input())
q = np.zeros((2**(n+1)))
q[1<<n] = 1

17


Шаг блоков адамара

In [348]:
for i in range(n+1):
  q = UnitGate(q,H,i)

Выбор одного из двух оракулов *constOracul* и *balanceOracul*

In [319]:
q = constOracul(q, n)

In [349]:
q = balanceOracul(q, n)

Шаг блоков адамара без последнего кубита

In [350]:
for i in range(n):
  q = UnitGate(q,H, i)

Измерение

In [351]:
q2 = q**2

In [352]:
indexnonzero = np.where(q2 > 0.001)
print((bin(indexnonzero[0][0])[2:].zfill(n+1))[1:n+1])
print((bin(indexnonzero[0][1])[2:].zfill(n+1))[1:n+1])

11111111111111111
11111111111111111


Если все 0 => константная
Если все 1 => сбалансированная